# Visualing portfolio performance

In [2]:
import sys

# Add the src directory to the path
# If it bugs out, just alternate between ../../ and ../../src
sys.path.append("../../")

from src.config import app_config
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

ENGINE  = create_engine(app_config.SQLALCHEMY_DATABASE_URI, echo=False, pool_size=20, max_overflow=0)

In [4]:
from src.service import DataService
from cryptolib.model import PortfolioHistoryModel, BalanceHistoryModel, CurrencyPairConfigModel
import matplotlib.pyplot as plt

def bot_info(bot: CurrencyPairConfigModel, **params):
    strat_config = ','.join([sc.value for sc in bot.strategy_config])
    output = f"Bot: {bot.id} - {bot.currency_pair} - {bot.strategy.value} ({strat_config}) - {bot.interval.value}"
    for key, val in params.items():
        output += f" - {key}: {val}"
    print(output)

### Bokeh
For nice graphing...

In [11]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.palettes import Category10, Dark2_5
from bokeh.io import output_notebook

import itertools

output_notebook()

colors = itertools.cycle(Dark2_5)

class BokehGraph:

    WIDTH = 1400
    HEIGHT = 600

    def __init__(self):
        self.figure = figure(width=self.WIDTH , height=self.HEIGHT, x_axis_type="datetime")
        self.figure.add_tools(HoverTool(
            tooltips=[
                ('symbol', '$name'),
                ('date', '@x{%F}'),
                ('value', '@y{0.0000}'),
            ],
            formatters={
                '@x': 'datetime',
            },
            mode='vline'
        ))

    def add_line(self, x, y, legend_label, name):
        source = ColumnDataSource(data=dict(x=x, y=y))
        self.figure.line(x="x", y="y", source=source, line_width=2, legend_label=legend_label, color=next(colors), name=name)

    def show(self):
        # Hide lines on click
        self.figure.legend.click_policy = "hide"
        show(self.figure)

Loading BokehJS ...

### Bot performance

In [6]:
# Number of active bots
with Session(ENGINE) as session:
    bots = DataService().get_active_bots(session)
    print(len(bots))

219


##### Show profitable bots

In [6]:
from src.service import DataService
from cryptolib.model import CurrencyPairConfigModel, OrderModel
from cryptolib.enums import OrderSide

with Session(ENGINE) as session:
    users = DataService().get_users(session)
    for user in users:
        bots = user.currency_pair_configs
        profit = 0
        for bot in bots:
            orders = bot.orders
            if len(orders) == 0:
                continue

            latest_order: OrderModel = orders[-1]            
            if latest_order.side.value == OrderSide.BUY.value:
                profit += latest_order.cost - bot.allocated_balance
            else:
                profit += latest_order.amount - bot.allocated_balance
        
        print(f"User Id: {user.id} | Strategy: {bots[0].strategy.value} | Interval: {bots[0].interval.value} - P/L: {profit}") if profit > 0 else None

User Id: 1 | Strategy: Bollinger Bands | Interval: 1m - P/L: 3.7841064699999833
User Id: 23 | Strategy: Relative Strength Index | Interval: 1m - P/L: 1284.6941351899995
User Id: 24 | Strategy: Chaikin Oscillator | Interval: 1m - P/L: 536.4037856599989
User Id: 26 | Strategy: Chaikin Oscillator | Interval: 5m - P/L: 503.4855538900001
User Id: 28 | Strategy: Chaikin Oscillator | Interval: 30m - P/L: 1104.3865486800005
User Id: 32 | Strategy: Commodity Channel Index | Interval: 1m - P/L: 761.3701511300005
User Id: 33 | Strategy: Commodity Channel Index | Interval: 5m - P/L: 879.4432600900009
User Id: 35 | Strategy: Commodity Channel Index | Interval: 30m - P/L: 778.8459996800002
User Id: 36 | Strategy: Commodity Channel Index | Interval: 1h - P/L: 967.9799149899991
User Id: 37 | Strategy: Commodity Channel Index | Interval: 2h - P/L: 125.2463946099997
User Id: 44 | Strategy: Chaikin Oscillator | Interval: 15m - P/L: 261.16311701999985
User Id: 45 | Strategy: Chaikin Oscillator | Interval:

**For a particular user**

In [17]:
from cryptolib.model import CurrencyPairConfigModel, OrderModel
from cryptolib.enums import OrderSide

# The user to check
# be mindful of how many active bots are running.
# E.g. plotting 400 bots is pretty slow
user_id = 28

with Session(ENGINE) as session:
    user = DataService().get_user(session, user_id)
    bots: list[CurrencyPairConfigModel] = user.currency_pair_configs
    
    print("=====================================")
    print(f"User: {user.email}")
    
    # Plot the history of orders for each bot in bokeh
    graph = BokehGraph()
    for i, bot in enumerate(bots):
        orders: list[OrderModel] = bot.orders
        x = [o.created_at for o in orders if o.side.value == OrderSide.BUY.value]
        y = [o.cost for o in orders if o.side.value == OrderSide.BUY.value]
        
        divisor = len(y) if len(y) > 0 else 1
        avg = sum(y) / divisor

        # Skip plotting the losing bots
        if avg < bot.allocated_balance:
            # Disable the bots if desired
            # bot.is_active = False
            # session.commit()
            continue
        
        bot_info(bot, avg=avg)
        graph.add_line(x, y, f"Bot {i} - {bot.currency_pair}", name=f"{bot.currency_pair}")

    graph.show()


User: bot28@admin.com
Bot: 136 - BNBUSDT - Chaikin Oscillator (11,15) - 30m - avg: 5055.225478370311
Bot: 132 - BTCUSDT - Chaikin Oscillator (11,15) - 30m - avg: 5086.15357205889
Bot: 133 - ETHUSDT - Chaikin Oscillator (11,15) - 30m - avg: 5085.510041094634
Bot: 134 - IMXUSDT - Chaikin Oscillator (11,15) - 30m - avg: 5237.280254711514
Bot: 135 - XRPUSDT - Chaikin Oscillator (11,15) - 30m - avg: 5063.601976732306
